In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
import numpy as np
from argparse import ArgumentParser
from double_net.double_net import DoubleNet, train_loop, train_loop_no_lagrange, test_loop, test_loop_random_start
from double_net import datasets as ds
import time
import double_net.plot_utils as pu
import importlib
import matplotlib.pyplot as plt

In [2]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [3]:
parser = ArgumentParser()
parser.add_argument('--random-seed', type=int, default=0)
parser.add_argument('--num-examples', type=int, default=131072 * 4)
parser.add_argument('--test-num-examples', type=int, default=1000)
parser.add_argument('--test-iter', type=int, default=5)
parser.add_argument('--n-agents', type=int, default=2)
parser.add_argument('--n-items', type=int, default=15)
parser.add_argument('--num-epochs', type=int, default=100)
parser.add_argument('--batch-size', type=int, default=128 * 32)
parser.add_argument('--test-batch-size', type=int, default=1000)
parser.add_argument('--model-lr', type=float, default=1e-3)
parser.add_argument('--misreport-lr', type=float, default=1e-1)
parser.add_argument('--misreport-iter', type=int, default=25)
parser.add_argument('--test-misreport-iter', type=int, default=1000)
parser.add_argument('--rho', type=float, default=1)
parser.add_argument('--rho-incr-iter', type=int, default=2)
parser.add_argument('--rho-incr-amount', type=float, default=1.0)
parser.add_argument('--lagr-update-iter', type=int, default=100)
parser.add_argument('--rgt-start', type=int, default=0)
parser.add_argument('--sinkhorn-epsilon', type=float, default=5e-2)
parser.add_argument('--sinkhorn-tolerance', type=float, default=0.01)


args = parser.parse_args(args=[])
args

Namespace(batch_size=4096, lagr_update_iter=100, misreport_iter=25, misreport_lr=0.1, model_lr=0.001, n_agents=2, n_items=15, num_epochs=100, num_examples=524288, random_seed=0, rgt_start=0, rho=1, rho_incr_amount=1.0, rho_incr_iter=2, sinkhorn_epsilon=0.05, sinkhorn_rounds=20, test_batch_size=1000, test_iter=5, test_misreport_iter=1000, test_num_examples=1000)

In [4]:
torch.manual_seed(args.random_seed)
np.random.seed(args.random_seed)

item_ranges = ds.preset_valuation_range(args.n_agents, args.n_items, "")

model = DoubleNet(
    args.n_agents, args.n_items, item_ranges, args.sinkhorn_epsilon, args.sinkhorn_tolerance, marginal_choice='unit'
).to(device)

train_data = ds.generate_dataset_nxk(args.n_agents, args.n_items, args.num_examples, item_ranges).to(device)
train_loader = ds.Dataloader(train_data, batch_size=args.batch_size, shuffle=True)

In [5]:
%%time
mean_rgt, mean_pay, lagr_mults = train_loop(model, train_loader, args, device=device)

  1%|          | 1/100 [01:05<1:47:58, 65.43s/it]

{'regret_max': 0.5126627683639526, 'regret_mean': 0.038296401500701904, 'regret_mults': tensor([[6.7697, 6.7880]], device='cuda:0'), 'payment': 0.8730233907699585}


  2%|▏         | 2/100 [02:10<1:46:48, 65.39s/it]

{'regret_max': 0.13299113512039185, 'regret_mean': 0.018997671082615852, 'regret_mults': tensor([[8.7282, 8.8623]], device='cuda:0'), 'payment': 1.1199986934661865}


  3%|▎         | 3/100 [03:15<1:45:38, 65.34s/it]

...

{'regret_max': 0.02804628014564514, 'regret_mean': 0.0006132174748927355, 'regret_mults': tensor([[318.2228, 223.8218]], device='cuda:0'), 'payment': 0.8451941013336182}


 99%|█████████▉| 99/100 [1:46:26<01:03, 63.69s/it]

{'regret_max': 0.009754776954650879, 'regret_mean': 0.00022779843129683286, 'regret_mults': tensor([[319.1113, 225.6398]], device='cuda:0'), 'payment': 0.8812506198883057}


100%|██████████| 100/100 [1:47:29<00:00, 64.50s/it]

{'regret_max': 0.009842455387115479, 'regret_mean': 0.000235067680478096, 'regret_mults': tensor([[321.1943, 229.6456]], device='cuda:0'), 'payment': 0.8878908157348633}
CPU times: user 1h 47min 10s, sys: 12 s, total: 1h 47min 22s
Wall time: 1h 47min 29s


In [6]:
dir_name = 'unit_2x15_0_1_experiment_data/'

In [7]:
%%time
# model.sinkhorn_rounds = 100
# model.sinkhorn_epsilon = 1e-2
test_data = ds.generate_dataset_nxk(args.n_agents, args.n_items, args.test_num_examples, item_ranges).to(device)
torch.save(test_data, dir_name + 'test_data')
#test_data = torch.load(dir_name + 'test_data').to(device=device)
cpu_test_data = test_data.clone().to(device='cpu')

test_loader = ds.Dataloader(test_data, batch_size=args.test_batch_size, shuffle=True)

result = test_loop(model, test_loader, args, device=device)
result

1it [00:18, 18.65s/it]

CPU times: user 18.6 s, sys: 23 ms, total: 18.6 s
Wall time: 18.7 s


{'payment_mean': 0.886029839515686,
 'regret_mean': 0.0012264503166079521,
 'regret_max': 0.008139997720718384}

In [17]:
%%time
args.test_num_examples = 1000
args.test_batch_size = 1000
args.test_misreport_iter = 1000
args.misreport_lr = 1e-1
model.sinkhorn_rounds = 20
model.sinkhorn_epsilon = 5e-2

random_starts = [test_data]
for i in range(10):
    random_starts.append(ds.generate_dataset_nxk(args.n_agents, args.n_items, args.test_num_examples, item_ranges).to(device))
test_loader = ds.Dataloader(test_data, batch_size=args.test_batch_size, shuffle=True)

result = test_loop_random_start(model, test_loader, args, random_starts, device=device)
result

1it [03:25, 205.94s/it]

CPU times: user 3min 25s, sys: 391 ms, total: 3min 25s
Wall time: 3min 25s


{'payment_mean': 0.886029839515686,
 'regret_mean': 0.0012776502408087254,
 'regret_max': 0.008141130208969116}

In [9]:
model.save(dir_name)

In [10]:
import cvxpy as cp
import numpy as np
from typing import NamedTuple
from tqdm.notebook import tqdm

In [11]:
class AuctionResult(NamedTuple):
    alloc: np.ndarray
    welfare: np.ndarray
    payment: np.ndarray

In [12]:
def max_welfare_alloc(bids_mat, k, exact_demand=False):
    # returns allocation, util of allocation

    num_agents = bids_mat.shape[0]
    num_items = bids_mat.shape[1]
    alloc_matrix = cp.Variable((num_agents, num_items), nonneg=True)
    item_supply_constraints = cp.sum(alloc_matrix, axis=0) <= 1
    if exact_demand:
        agent_demand_constraints = cp.sum(alloc_matrix, axis=1) == k
    else:
        agent_demand_constraints = cp.sum(alloc_matrix, axis=1) <= k
    welfare = cp.sum((cp.multiply(bids_mat, alloc_matrix)))
    problem = cp.Problem(cp.Maximize(welfare), [item_supply_constraints, agent_demand_constraints])
    problem.solve()
    return (alloc_matrix.value, problem.value)

In [13]:
def vcg_auction(bids_mat, k, exact_demand=False):
    main_alloc, max_welfare_total_util = max_welfare_alloc(bids_mat, k, exact_demand=exact_demand)
    payments = np.zeros(bids_mat.shape[0])
    player_utils = (bids_mat * main_alloc).sum(axis=1)
    num_agents = bids_mat.shape[0]
    for i in range(num_agents):
        dropped_bid_mat = np.delete(bids_mat, (i), axis=0)
        dropped_player_utils = np.delete(player_utils, (i), axis=0) # player utils under full auction
        new_alloc, new_total_util = max_welfare_alloc(dropped_bid_mat, k, exact_demand=exact_demand)
        new_agent_utils = (new_alloc*dropped_bid_mat).sum(axis=1) # player utils without agent i's bid
        payments[i] = (new_agent_utils - dropped_player_utils).sum() 
    return AuctionResult(main_alloc, player_utils, payments)

In [14]:
payments = []
for i in range(test_data.shape[0]):
    payments.append(vcg_auction(cpu_test_data[i].cpu().numpy(), 1, exact_demand=False).payment.sum())

In [15]:
np.mean(payments)

0.0018617426402055984

In [16]:
import pickle
to_pkl_lst = [mean_rgt, mean_pay, lagr_mults]
for i, fname in enumerate(['mean_rgt', 'mean_pay', 'lagr_mults']):
    with open(dir_name + fname, 'wb') as fp:
        pickle.dump(to_pkl_lst[i], fp)